Q3 (scatterplot)
idea 1:
- scatterplot
- eix x: school
- eix y: mass shootings
- color: year
- point: state in year

idea2:
- violin plot
- violin: state
- distribucio violin: years

idea3:
- scatterplot de totals, sense separar per anys


county, state, shootings, school, year

In [25]:
import pandas as pd
import altair as alt

df_shootings = pd.read_csv("datasets/MassShootingsComplete_FIPS.csv")
df_school_incidents = pd.read_csv("datasets/SchoolIncidents_FIPS.csv")
df_population = pd.read_csv("datasets/CountyPopulationAllYears.csv")

In [ ]:

total_shootings_per_state = df_shootings.groupby('State').size().reset_index(name='Total Shootings')


# Calcula el total d'incidents escolars per estat
total_school_incidents_per_state = df_school_incidents.groupby('State').size().reset_index(name='Total School Incidents')

# Uneix els dos datasets basant-te en l'estat
merged_data = pd.merge(total_shootings_per_state, total_school_incidents_per_state, on='State')

# Crea el scatterplot amb Altair
scatterplot = alt.Chart(merged_data).mark_circle(size=100).encode(
    x=alt.Y('Total Shootings:Q', title='Total Shootings'),
    y=alt.X('Total School Incidents:Q', title='Total School Incidents'),
    tooltip=['State', 'Total Shootings', 'Total School Incidents'],  # Afegeix informació de descripció
).properties(
    title='Comparison of Total Shootings and School Incidents by State'
)

scatterplot.show()

alt.Chart(...)

In [30]:
df_population.head()

,STNAME,CTYNAME,FIPS,STATE,COUNTY,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE2020,POPESTIMATE2021,POPESTIMATE2022,POPESTIMATE2023
0,Alabama,Alabama,1000,1,0,4843737.0,4854803.0,4866824.0,4877989.0,4891628.0,4907965.0,5031864.0,5050380.0,5073903.0,5108468.0
1,Alabama,Autauga County,1001,1,1,54922.0,54903.0,55302.0,55448.0,55533.0,55769.0,58915.0,59203.0,59726.0,60342.0
2,Alabama,Baldwin County,1003,1,3,199306.0,203101.0,207787.0,212737.0,218071.0,223565.0,233227.0,239439.0,246531.0,253507.0
3,Alabama,Barbour County,1005,1,5,26768.0,26300.0,25828.0,25169.0,24887.0,24657.0,24969.0,24533.0,24700.0,24585.0
4,Alabama,Bibb County,1007,1,7,22541.0,22553.0,22590.0,22532.0,22300.0,22313.0,22188.0,22359.0,21986.0,21868.0


In [ ]:
# Agrupar per estat i comptar el nombre total de tirotejos per estat
total_shootings_per_state 

# Agrupar per estat i obtenir la població total de cada estat
df_population_state = df_population.groupby('STNAME')['POPESTIMATE2020'].sum().reset_index()
df_population_state['State'] = df_population_state['STNAME']
df_school_incidents_state = df_school_incidents.groupby('State').size().reset_index(name='Total School Incidents')

# Unir les dades de tirotejos amb les dades de població pel mateix estat
df_combined = pd.merge(total_shootings_per_state, df_population_state, on='State')
df_combined = pd.merge(df_combined, df_school_incidents_state, on='State')
df_combined['School Incident Ratio'] = df_combined['Total School Incidents'] / df_combined['POPESTIMATE2020']

# Calcular el ratio de tirotejos per població
df_combined['Ratio'] = df_combined['Total Shootings'] / df_combined['POPESTIMATE2020']

# Crear el scatter plot
scatter_plot = alt.Chart(df_combined).mark_circle(size=100).encode(
    y=alt.Y('Ratio:Q', title='Ratio of Shootings per Population'),
    x=alt.X('School Incident Ratio:Q', sort='-x', title=''),
    color=alt.Color('Ratio:Q', scale=alt.Scale(scheme='viridis'), title='Ratio'),
    tooltip=['State', 'Total Shootings', 'POPESTIMATE2020', 'Ratio']
).properties(
    width=800,
    height=600,
    title="Ratio of Shootings per Population by State"
)

scatter_plot.show()


KeyError: 'Year'

In [14]:
# Converteix les dates a un format de data de pandas

df_shootings['Incident Date'] = pd.to_datetime(df_shootings['Incident Date'])
df_school_incidents['Incident Date'] = pd.to_datetime(df_school_incidents['Incident Date'])

# Extreu l'any de la data de l'incident per ambdós datasets
df_shootings['Year'] = df_shootings['Incident Date'].dt.year
df_school_incidents['Year'] = df_school_incidents['Incident Date'].dt.year

# Filtra només els anys 2022 i posteriors
df_shootings = df_shootings[df_shootings['Year'] >= 2022]
df_school_incidents = df_school_incidents[df_school_incidents['Year'] >= 2022]

# Calcula el total de tirotejos per estat i any
total_shootings_per_state_year = df_shootings.groupby(['State', 'Year']).size().reset_index(name='Total Shootings')

# Calcula el total d'incidents escolars per estat i any
total_school_incidents_per_state_year = df_school_incidents.groupby(['State', 'Year']).size().reset_index(name='Total School Incidents')

# Uneix els dos datasets per estat i any
merged_data = pd.merge(total_shootings_per_state_year, total_school_incidents_per_state_year, on=['State', 'Year'])

# Crea el scatterplot amb Altair
scatterplot = alt.Chart(merged_data).mark_circle(size=100).encode(
    x=alt.X('Total Shootings:Q', title='Total Shootings', scale=alt.Scale(type='log')),
    y=alt.Y('Total School Incidents:Q', title='Total School Incidents', scale=alt.Scale(type='log')),
    color=alt.Color('Year:O', 
                    title='Year', 
                    scale=alt.Scale(
                        domain=[2022, 2023],  # Anys que tens en les dades
                        range=['#f1c40f', '#9b59b6']  # Paleta de colors contrastada
                    )
    ),
    tooltip=['State', 'Year', 'Total Shootings', 'Total School Incidents'],  # Afegeix informació de descripció
).properties(
    title='Comparison of Total Shootings and School Incidents by State (2022 onwards)'
)


scatterplot.show()

alt.Chart(...)